# 第13章：Agent Structured Output（结构化输出）

## 学习目标

本章将学习：
1. Agent结构化输出的概念和优势
2. `response_format`参数的使用
3. ToolStrategy（工具调用策略）
4. ProviderStrategy（原生提供商策略）
5. 错误处理与验证
6. 动态选择输出格式
7. 实战：智能数据提取Agent

## 为什么Agent需要结构化输出？

**问题**：Agent的默认输出是自然语言文本，难以：
- 直接用于数据库存储
- 与其他系统集成
- 进行自动化处理

**解决方案**：让Agent返回**结构化数据**（JSON、Pydantic模型）

---

### 对比：普通输出 vs 结构化输出

| 普通输出 | 结构化输出 |
|---------|-----------|
| "联系人：张三，邮箱：..." | `ContactInfo(name="张三", ...)` |
| 需要正则解析 | 直接使用对象 |
| 容易出错 | 自动验证 |
| 不可预测 | 结构固定 |

---

## 核心概念

### Agent结构化输出流程

```
User Query
    ↓
Agent执行（可能调用工具）
    ↓
Agent完成工具调用
    ↓
【生成结构化输出】← response_format
    ↓
返回：result["structured_response"]
```

**关键点**：
- 结构化输出在Agent完成**所有工具调用后**生成
- 不影响Agent的推理和工具调用过程
- 在`structured_response`字段中获取

In [1]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import create_agent
from pydantic import BaseModel, Field

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")

环境配置完成！
模型: gpt-4.1-mini


## 1. 基础结构化输出

最简单的方式：直接传入Pydantic模型作为`response_format`

In [2]:
print("【演示：基础结构化输出】")
print()

# 定义输出schema
class ContactInfo(BaseModel):
    """联系人信息"""
    name: str = Field(description="姓名")
    email: str = Field(description="邮箱地址")
    phone: str = Field(description="电话号码")

# 创建Agent（无工具）
agent_basic = create_agent(
    model=model,
    tools=[],
    response_format=ContactInfo,  # 直接传入Pydantic模型
    system_prompt="从用户输入中提取联系人信息。"
)

# 测试
response = agent_basic.invoke({
    "messages": [{"role": "user", "content": "我是张三，邮箱是zhangsan@example.com，电话是13800138000"}]
})

print("【Agent的文本回答】")
print(response['messages'][-1].content)
print()

print("【结构化输出】")
structured = response['structured_response']
print(f"类型: {type(structured)}")
print(f"内容: {structured}")
print()

print("【直接使用对象属性】")
print(f"姓名: {structured.name}")
print(f"邮箱: {structured.email}")
print(f"电话: {structured.phone}")
print()

print("说明: Agent在完成推理后，自动将结果转换为Pydantic对象")

【演示：基础结构化输出】

【Agent的文本回答】
{"name":"张三","email":"zhangsan@example.com","phone":"13800138000"}

【结构化输出】
类型: <class '__main__.ContactInfo'>
内容: name='张三' email='zhangsan@example.com' phone='13800138000'

【直接使用对象属性】
姓名: 张三
邮箱: zhangsan@example.com
电话: 13800138000

说明: Agent在完成推理后，自动将结果转换为Pydantic对象


## 2. ToolStrategy vs ProviderStrategy

Agent可以用两种策略生成结构化输出：

### ToolStrategy（工具策略）
- **原理**：通过"人造工具调用"生成结构化输出
- **优势**：兼容所有支持工具调用的模型
- **适用**：通用场景

### ProviderStrategy（提供商策略）
- **原理**：使用模型提供商的原生结构化输出API
- **优势**：更可靠、更准确
- **限制**：仅OpenAI、Anthropic、Grok等部分模型支持
- **适用**：生产环境、高准确率需求

---

### 自动选择

直接传入Pydantic模型时，LangChain会自动选择：
- 如果模型支持原生结构化输出 → `ProviderStrategy`
- 否则 → `ToolStrategy`

也可以手动指定策略

In [3]:
print("【演示：显式使用ToolStrategy】")
print()

from langchain.agents.structured_output import ToolStrategy, ProviderStrategy

# 定义输出schema
class Weather(BaseModel):
    """天气信息"""
    temperature: float = Field(description="温度（摄氏度）")
    condition: str = Field(description="天气状况，如：晴天、多云、雨天")
    city: str = Field(description="城市名称")

# 定义工具
@tool
def get_weather(city: str) -> str:
    """获取指定城市的天气"""
    weather_data = {
        "北京": "晴天，温度25°C",
        "上海": "多云，温度28°C",
        "深圳": "雨天，温度30°C"
    }
    return weather_data.get(city, "未知城市")

# 使用ToolStrategy
agent_tool_strategy = create_agent(
    model=model,
    tools=[get_weather],
    response_format=ToolStrategy(Weather),  # 显式使用ToolStrategy
    system_prompt="查询城市天气并返回结构化结果。"
)

print("【测试ToolStrategy】")
response = agent_tool_strategy.invoke({
    "messages": [{"role": "user", "content": "查询北京的天气"}]
})

structured = response['structured_response']
print(f"结构化输出: {structured}")
print(f"  城市: {structured.city}")
print(f"  温度: {structured.temperature}°C")
print(f"  状况: {structured.condition}")
print()
print("说明: ToolStrategy通过人造工具调用实现结构化输出，兼容性好")

【演示：显式使用ToolStrategy】

【测试ToolStrategy】
结构化输出: temperature=25.0 condition='晴天' city='北京'
  城市: 北京
  温度: 25.0°C
  状况: 晴天

说明: ToolStrategy通过人造工具调用实现结构化输出，兼容性好


### 2.1 ProviderStrategy示例

In [4]:
print("【演示：使用ProviderStrategy】")
print()

# ProviderStrategy需要模型原生支持结构化输出
# OpenAI的gpt-4o/gpt-4o-mini支持原生结构化输出

try:
    agent_provider_strategy = create_agent(
        model=model,
        tools=[],
        response_format=ProviderStrategy(ContactInfo),  # 使用ProviderStrategy
        system_prompt="从用户输入中提取联系人信息。"
    )
    
    response = agent_provider_strategy.invoke({
        "messages": [{"role": "user", "content": "联系李四，邮箱lisi@test.com，手机15900159000"}]
    })
    
    structured = response['structured_response']
    print(f"结构化输出: {structured}")
    print("说明: ProviderStrategy使用模型原生API，更可靠")
except Exception as e:
    print(f"注意: 当前模型可能不支持ProviderStrategy")
    print(f"错误: {str(e)[:100]}")
    print("建议: 使用ToolStrategy或切换到支持原生结构化输出的模型（如gpt-4o）")

【演示：使用ProviderStrategy】

结构化输出: name='李四' email='lisi@test.com' phone='15900159000'
说明: ProviderStrategy使用模型原生API，更可靠


## 3. 错误处理与验证

当Agent生成的数据不符合schema时，可以配置错误处理策略

### handle_errors参数

| 值 | 行为 |
|----|-----|
| `True`（默认） | 自动重试，让LLM看到错误并改正 |
| `False` | 抛出异常 |
| `str` | 自定义错误提示 |
| `Exception类型` | 只处理特定类型的错误 |
| `callable` | 自定义错误处理函数 |

In [5]:
print("【演示：错误处理与验证】")
print()

# 定义带验证规则的schema
class ProductRating(BaseModel):
    """产品评分"""
    rating: int = Field(description="评分，必须在1-5之间", ge=1, le=5)
    comment: str = Field(description="评论内容")
    product_name: str = Field(description="产品名称")

print("【测试1：正常情况】")
agent_validation = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(ProductRating),  # 默认handle_errors=True
    system_prompt="解析产品评价，rating必须在1-5之间。"
)

response1 = agent_validation.invoke({
    "messages": [{"role": "user", "content": "这款手机非常棒！我给4分"}]
})
print(f"结果: {response1['structured_response']}")
print()

print("【测试2：验证失败情况（rating超出范围）】")
# 这里LLM会自动重试修正
response2 = agent_validation.invoke({
    "messages": [{"role": "user", "content": "这产品太好了，我给10分！"}]
})
structured = response2['structured_response']
print(f"结果: {structured}")
print(f"注意: rating={structured.rating}，Agent自动将10调整为5（范围内最大值）")
print()

print("【测试3：禁用错误处理】")
agent_no_handling = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(ProductRating, handle_errors=False),
    system_prompt="解析产品评价。"
)

try:
    response3 = agent_no_handling.invoke({
        "messages": [{"role": "user", "content": "给20分"}]
    })
    print(f"结果: {response3['structured_response']}")
except Exception as e:
    print(f"错误被抛出: {type(e).__name__}")
    print("说明: handle_errors=False时，验证失败会抛出异常")

【演示：错误处理与验证】

【测试1：正常情况】
结果: rating=4 comment='这款手机非常棒！' product_name='手机'

【测试2：验证失败情况（rating超出范围）】
结果: rating=5 comment='这产品太好了，我给10分！' product_name='未知产品'
注意: rating=5，Agent自动将10调整为5（范围内最大值）

【测试3：禁用错误处理】
结果: rating=5 comment='给20分' product_name='未知产品'


## 4. 复杂嵌套结构

结构化输出支持嵌套的Pydantic模型

In [6]:
print("【演示：嵌套结构】")
print()

from typing import List

# 定义嵌套模型
class Address(BaseModel):
    """地址信息"""
    street: str = Field(description="街道")
    city: str = Field(description="城市")
    country: str = Field(description="国家")

class Person(BaseModel):
    """人员信息（包含嵌套的地址）"""
    name: str = Field(description="姓名")
    age: int = Field(description="年龄")
    addresses: List[Address] = Field(description="地址列表")
    email: str = Field(description="邮箱")

# 创建Agent
agent_nested = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(Person),
    system_prompt="从文本中提取人员信息，包括所有地址。"
)

# 测试
text = """
张伟，35岁，邮箱zhangwei@example.com。
他有两个地址：
1. 北京市朝阳区建国路88号，中国
2. 上海市浦东新区世纪大道100号，中国
"""

response = agent_nested.invoke({
    "messages": [{"role": "user", "content": text}]
})

person = response['structured_response']
print(f"姓名: {person.name}")
print(f"年龄: {person.age}")
print(f"邮箱: {person.email}")
print(f"地址数量: {len(person.addresses)}")
print()
print("地址详情:")
for i, addr in enumerate(person.addresses, 1):
    print(f"  地址{i}: {addr.street}, {addr.city}, {addr.country}")

print()
print("说明: 支持复杂的嵌套结构和列表字段")

【演示：嵌套结构】

姓名: 张伟
年龄: 35
邮箱: zhangwei@example.com
地址数量: 2

地址详情:
  地址1: 朝阳区建国路88号, 北京市, 中国
  地址2: 浦东新区世纪大道100号, 上海市, 中国

说明: 支持复杂的嵌套结构和列表字段


## 5. 动态选择输出格式

使用Middleware根据对话状态、用户角色等动态选择输出格式

In [14]:
print("【演示：动态输出格式】")
print()

from langchain.agents.middleware import wrap_model_call

# 定义两种输出格式
class SimpleResponse(BaseModel):
    """简单回答"""
    answer: str = Field(description="简短回答")

class DetailedResponse(BaseModel):
    """详细回答"""
    answer: str = Field(description="详细回答")
    reasoning: str = Field(description="推理过程")
    confidence: float = Field(description="置信度(0-1)", ge=0, le=1)

# 创建动态选择middleware
@wrap_model_call
def dynamic_output_format(request, handler):
    """根据消息数量选择输出格式"""
    message_count = len(request.messages)
    
    if message_count < 3:
        # 早期对话：简单格式
        request = request.override(response_format=SimpleResponse)
        print(f"  [Middleware] 消息数={message_count}，使用SimpleResponse")
    else:
        # 深入对话：详细格式
        request = request.override(response_format=DetailedResponse)
        print(f"  [Middleware] 消息数={message_count}，使用DetailedResponse")
    
    return handler(request)

# 创建Agent
agent_dynamic = create_agent(
    model=model,
    tools=[],
    response_format=SimpleResponse,
    middleware=[dynamic_output_format],
    system_prompt="回答用户问题。"
)

# 测试1：第一条消息（简单格式）
print("【测试1：第一次对话】")
response1 = agent_dynamic.invoke({
    "messages": [{"role": "user", "content": "什么是Python？"}]
})
result1 = response1['structured_response']
print(f"输出类型: {type(result1).__name__}")
print(f"内容: {result1}")
print()

# 测试2：多条消息（详细格式）
print("【测试2：深入对话】")
response2 = agent_dynamic.invoke({
    "messages": [
        {"role": "user", "content": "什么是Python？"},
        {"role": "assistant", "content": "Python是一种编程语言"},
        {"role": "user", "content": "它有什么特点？"}
    ]
})
result2 = response2['structured_response']
print(f"输出类型: {type(result2).__name__}")
print(f"答案: {result2.answer}")
if hasattr(result2, 'reasoning'):
    print(f"推理: {result2.reasoning}")
    print(f"置信度: {result2.confidence}")

print()
print("说明: Middleware可以根据上下文动态调整输出格式")

【演示：动态输出格式】

【测试1：第一次对话】
  [Middleware] 消息数=1，使用SimpleResponse


  [Middleware] 消息数=2，使用SimpleResponse
  [Middleware] 消息数=3，使用DetailedResponse


KeyboardInterrupt: 

## 6. 实战项目：智能数据提取Agent

构建一个能从非结构化文本中提取结构化信息的Agent

In [15]:
print("【实战项目：智能数据提取Agent】")
print()

from datetime import datetime
from typing import Optional

# 定义输出schema
class EventInfo(BaseModel):
    """事件信息"""
    title: str = Field(description="事件标题")
    date: Optional[str] = Field(description="事件日期（YYYY-MM-DD格式）", default=None)
    location: Optional[str] = Field(description="事件地点", default=None)
    participants: List[str] = Field(description="参与人员列表", default_factory=list)
    description: str = Field(description="事件描述")
    category: str = Field(description="事件类别：会议/培训/活动/其他")

# 定义辅助工具
@tool
def search_person_info(name: str) -> str:
    """查询人员信息"""
    people_db = {
        "张三": "技术总监，负责架构设计",
        "李四": "产品经理，负责需求管理",
        "王五": "开发工程师，负责后端开发"
    }
    return people_db.get(name, f"{name}的信息未找到")

# 创建数据提取Agent
extraction_agent = create_agent(
    model=model,
    tools=[search_person_info],
    response_format=ToolStrategy(
        EventInfo,
        handle_errors=True  # 自动处理验证错误
    ),
    system_prompt="""你是数据提取专家。从文本中提取事件信息：
1. 识别事件的标题、日期、地点
2. 提取所有参与人员
3. 如需了解人员背景，使用search_person_info工具
4. 根据内容判断事件类别
5. 总结事件描述"""
)

# 测试文本
test_texts = [
    """
    技术评审会定于2024年3月15日上午10点在会议室A举行。
    参会人员包括：张三、李四、王五。
    会议主要讨论新版本架构设计方案。
    """,
    """
    公司年会将在12月20日晚上7点在希尔顿酒店宴会厅举办。
    全体员工参加，有抽奖和表演环节。
    """,
    """
    Python培训课程安排在下周三下午2点开始。
    地点：培训室B。讲师：张三。
    内容涵盖Python基础和数据分析。
    """
]

# 批量提取
for i, text in enumerate(test_texts, 1):
    print(f"{'='*60}")
    print(f"【文本 {i}】")
    print(text.strip())
    print()
    
    response = extraction_agent.invoke({
        "messages": [{"role": "user", "content": f"请从以下文本中提取事件信息：\n{text}"}]
    })
    
    event = response['structured_response']
    
    print("【提取结果】")
    print(f"标题: {event.title}")
    print(f"日期: {event.date or '未指定'}")
    print(f"地点: {event.location or '未指定'}")
    print(f"类别: {event.category}")
    print(f"参与人员: {', '.join(event.participants) if event.participants else '未指定'}")
    print(f"描述: {event.description}")
    print()

print("="*60)
print("✓ 智能数据提取Agent演示完成")
print()
print("【应用场景】")
print("- 会议记录自动整理")
print("- 邮件信息提取")
print("- 文档结构化处理")
print("- 日程管理自动化")

【实战项目：智能数据提取Agent】

【文本 1】
技术评审会定于2024年3月15日上午10点在会议室A举行。
    参会人员包括：张三、李四、王五。
    会议主要讨论新版本架构设计方案。

【提取结果】
标题: 技术评审会
日期: 2024-03-15
地点: 会议室A
类别: 会议
参与人员: 张三, 李四, 王五
描述: 技术评审会于2024年3月15日上午10点在会议室A举行，主要讨论新版本架构设计方案。参会人员包括技术总监张三、产品经理李四和开发工程师王五。

【文本 2】
公司年会将在12月20日晚上7点在希尔顿酒店宴会厅举办。
    全体员工参加，有抽奖和表演环节。

【提取结果】
标题: 公司年会
日期: 2023-12-20
地点: 希尔顿酒店宴会厅
类别: 活动
参与人员: 全体员工
描述: 公司年会将在12月20日晚上7点在希尔顿酒店宴会厅举办。全体员工参加，有抽奖和表演环节。

【文本 3】
Python培训课程安排在下周三下午2点开始。
    地点：培训室B。讲师：张三。
    内容涵盖Python基础和数据分析。

【提取结果】
标题: Python培训课程
日期: 未指定
地点: 培训室B
类别: 培训
参与人员: 张三
描述: Python培训课程安排在下周三下午2点开始，地点为培训室B。讲师为张三，内容涵盖Python基础和数据分析。

✓ 智能数据提取Agent演示完成

【应用场景】
- 会议记录自动整理
- 邮件信息提取
- 文档结构化处理
- 日程管理自动化


## 7. 总结与最佳实践

### 核心概念回顾

**1. 结构化输出的两种策略**

| 策略 | ToolStrategy | ProviderStrategy |
|------|-------------|------------------|
| 实现方式 | 人造工具调用 | 模型原生API |
| 兼容性 | 所有支持工具调用的模型 | 仅OpenAI、Anthropic等 |
| 可靠性 | 较高 | 更高 |
| 性能 | 正常 | 更快 |
| 使用场景 | 通用场景 | 生产环境、高准确率需求 |

**2. 获取结构化输出**

```python
response = agent.invoke({"messages": [...]})
structured = response['structured_response']  # Pydantic对象
```

**3. 错误处理策略**

```python
# 自动重试（默认）
ToolStrategy(Schema, handle_errors=True)

# 抛出异常
ToolStrategy(Schema, handle_errors=False)

# 自定义错误消息
ToolStrategy(Schema, handle_errors="请检查输入格式")
```

**4. 动态输出格式**

使用Middleware根据上下文动态调整：
```python
@wrap_model_call
def dynamic_format(request, handler):
    format = select_format(request.messages)  # 根据状态选择
    request = request.override(response_format=format)
    return handler(request)
```

---

### 与第4章对比

| 特性 | 第4章（Model结构化输出） | 第13章（Agent结构化输出） |
|------|------------------------|-------------------------|
| 适用对象 | 单个Model | 整个Agent（含工具） |
| 执行时机 | 每次Model调用 | Agent完成所有工具调用后 |
| 工具支持 | 无 | 可在结构化输出前调用工具 |
| 获取方式 | `model.with_structured_output()` | `create_agent(..., response_format=...)` |
| 返回位置 | 直接返回 | `result['structured_response']` |

---

### 最佳实践

**Schema设计：**
- 使用清晰的字段描述（`Field(description=...)`）
- 添加验证规则（`ge`, `le`, `pattern`等）
- 合理使用`Optional`和默认值
- 嵌套结构不要过深（建议≤3层）

**策略选择：**
- 开发阶段：直接传入Schema，让LangChain自动选择
- 生产环境且模型支持：优先使用`ProviderStrategy`
- 需要兼容性：使用`ToolStrategy`
- 多模型部署：根据模型动态选择策略

**错误处理：**
- 生产环境：开启`handle_errors=True`
- 调试阶段：使用`handle_errors=False`快速发现问题
- 关键数据：添加详细的验证规则
- 记录验证失败的情况用于改进Schema

**性能优化：**
- 结构化输出是在Agent完成后生成的，不影响工具调用
- 使用ProviderStrategy可以减少一次额外的LLM调用（相比v0.x）
- 简化Schema可以提高生成速度和准确率
- 避免在高频场景中使用过于复杂的嵌套结构

**与工具集成：**
- Agent可以在生成结构化输出前调用工具补充信息
- 工具的返回值可以用于填充结构化输出的字段
- 在`system_prompt`中明确说明何时使用工具

---

### 常见问题

**Q1: 为什么结构化输出字段不完整？**
- 检查Field描述是否清晰
- 在system_prompt中强调必填字段
- 使用验证规则确保数据完整性

**Q2: ProviderStrategy报错怎么办？**
- 确认模型是否支持原生结构化输出
- 降级使用ToolStrategy
- 检查模型版本是否最新

**Q3: 如何处理可选字段？**
- 使用`Optional[type]`
- 提供合理的默认值
- 在描述中说明何时该字段为空

**Q4: 结构化输出和普通输出可以同时返回吗？**
- 可以，`response['messages'][-1].content`是自然语言输出
- `response['structured_response']`是结构化输出
- 两者内容相关但格式不同

**Q5: 如何验证结构化输出的质量？**
- 使用Pydantic的验证规则自动检查
- 记录验证失败的案例
- 设置置信度字段让LLM自评
- 对关键字段进行二次验证